## Data Cleaning

In [1]:
import os
import sys
import warnings
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

### Events

In [5]:
def clean_events(df):
    res = df["location"].str.extract(r"(?:(?P<city>.*), )?(?P<state>.*), (?P<country>.*)")
    df = pd.concat([df, res], axis=1)
    
    df["city"] = df["city"].fillna("Unspecified")
    
    df["name"] = df["name"].astype("string")
    df["date"] = pd.to_datetime(df["date"], format="%B %d, %Y", errors="coerce")\
                .fillna(pd.to_datetime(df["date"], format="%b %d, %Y", errors="coerce"))
    df["id"] = df["url"].map(lambda s: os.path.split(s)[1])\
                        .map(lambda s: int(str(s), 16))\
                        .astype("uint")
    df["city"] = df["city"].astype("category")
    df["state"] = df["state"].astype("category")
    df["country"] = df["country"].astype("category")
    
    df = df.drop(["location", "url"], axis=1)
    
    df = df.rename({col:f"event_{col}" for col in df.columns}, axis=1)
    
    return df

In [6]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_events.csv")
completed_events = clean_events(pd.read_csv(filepath))
filepath = os.path.join(dir_dict["clean"], "completed_events.parquet")
completed_events.to_parquet(filepath)

In [7]:
filepath = os.path.join(dir_dict["raw_csv"], "upcoming_events.csv")
upcoming_events = clean_events(pd.read_csv(filepath))
filepath = os.path.join(dir_dict["clean"], "upcoming_events.parquet")
upcoming_events.to_parquet(filepath)

### Fights

In [8]:
filepath = os.path.join(dir_dict["raw_csv"], "completed_fights.csv")
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    df = pd.read_csv(filepath)

In [9]:
def join_weight_class(df):
    filepath = os.path.join(dir_dict["raw_csv"], "completed_fight_urls_weightclasses.csv")
    wc_df = pd.read_csv(filepath)
    wc_df["Fight ID"] = \
            wc_df["Fight Url"].map(lambda s: os.path.split(s)[1])
    df = df.merge(wc_df.drop("Fight Url", axis=1), on="Fight ID", how="left")
    
    return df

In [10]:
def clean_fights(df):
    
    df = join_weight_class(df) 
    
    to_drop = [col for col in df.columns if "Details:" in col] + ["Event Name"]
    df = df.drop(to_drop, axis=1)
    
    fighter1_cols = [col for col in df.columns if "Fighter1" in col]
    fighter2_cols = [col for col in df.columns if "Fighter2" in col]
    general_cols = [col for col in df.columns \
                            if col not in set(fighter1_cols).union(fighter2_cols)]
    
    df2 = df.copy(deep=True)

    df = df.drop(fighter2_cols, axis=1)
    df2 = df2.drop(fighter1_cols, axis=1)

    # df.columns = lmap(lambda col: col.replace("Fighter1_",""), df.columns)
    df.columns = lmap(lambda col: col.replace("Fighter1","Fighter"), df.columns)

    # df2.columns = lmap(lambda col: col.replace("Fighter2_",""), df2.columns)
    df2.columns = lmap(lambda col: col.replace("Fighter2","Fighter"), df2.columns)

    df = pd.concat([df, df2], axis=0)
    
    df.columns = lmap(lambda col: col.lower().replace(" ","_").replace(".",""), df.columns)
    df = df.drop("fighter_name", axis=1)
    
    coltrack = ColumnTracker(list(df.columns))

    df["fighter_won"] = (df["fighter_status"] == "W").astype("uint8")
    df = df.drop("fighter_status", axis=1)
    coltrack.remove(["fighter_status"])
    
    df = df.loc[df["fighter_overall_kd"].notnull()]
    
    # String columns
    str_cols = ["referee", "details"]
    df[str_cols] = df[str_cols].astype("string")
    coltrack.remove(str_cols)
    
    
    # ID columns
    url_cols = [col for col in df.columns if "url" in col]
    new_id_cols = lmap(lambda s: s.replace("url", "id"), url_cols)
    df[new_id_cols] = df[url_cols].applymap(lambda s: os.path.split(s)[1])
    
    id_cols = [col for col in df.columns if "id" in col]
    df.loc[:, id_cols] = df.loc[:,id_cols].applymap(lambda s: int(str(s), 16)).astype("uint")
    
    df = df.drop(url_cols, axis=1)
    coltrack.remove(["fight_id"] + url_cols)
    
    
    # Category columns
    cat_cols = ["bout", "method", "time_format", "weight_class"]
    df[cat_cols] = df[cat_cols].astype("category")
    coltrack.remove(cat_cols)
    
    
    # Percentage columns
    is_pct_col = df.select_dtypes("object").apply(lambda s: s.str.contains("%")).any()
    pct_cols = is_pct_col[is_pct_col == True].index.to_list()
    df[pct_cols] = df[pct_cols].apply(lambda s: \
                                        s.str.replace("%","").astype("float") / 100)
    coltrack.remove(pct_cols)
    
    
    # Out-of columns
    is_outof_col = df.select_dtypes("object").apply(lambda s: \
                                                    s.str.contains(r"\d+ of \d+")).any()
    outof_cols = is_outof_col[is_outof_col == True].index.to_list()

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for col in outof_cols:
            df = pd.concat([df, \
                            df[col].str.extract(f"(?P<{col}_landed>\d+) of (?P<{col}_total>\d+)")],
                           axis=1)

    df = df.drop(columns=outof_cols)
    coltrack.remove(outof_cols)
    
    
    # Time columns
    is_time_col = df.select_dtypes("object").apply(lambda s: s.str.contains(r"\d+:\d+")).any()
    time_cols = is_time_col[is_time_col == True].index.to_list()

    def seconds_extrator(row):
        return row[0]*60 + row[1]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for col in time_cols:
            df[f"{col}_seconds"] = \
                        df[col].str.extract(r"(\d+):(\d+)").astype("float").apply(seconds_extrator, axis=1)
        
    df = df.drop(columns=time_cols)
    coltrack.remove(time_cols)
    
    
    # Float columns
    float_cols = df.select_dtypes("float").columns.to_list()
    df[float_cols] = df[float_cols].astype("float")
    coltrack.remove(float_cols)
    
    
    # Round column
    df["round"] = df["round"].astype("uint8")
    coltrack.remove(["round"])
    
    rename_dict = {col:f"fight_{col}" \
                   for col in df.columns \
                   if all([prefix not in col for prefix in ["event_", "fight_"]])
                      and col != "fighter_id"}
    
    df = df.rename(rename_dict, axis=1)
    
    return df

In [11]:
df = clean_fights(df)
filepath = os.path.join(dir_dict["clean"], "completed_fights.parquet")
df.to_parquet(filepath)

Remaining columns: 114
Remaining columns: 113
Remaining columns: 111
Remaining columns: 108
Remaining columns: 104
Remaining columns: 86
Remaining columns: 26
Remaining columns: 19
Remaining columns: 1
Remaining columns: 0


In [12]:
df.head()

,fight_id,fight_bout,fight_method,fight_round,fight_time_format,fight_referee,fight_details,fight_fighter_overall_kd,fight_fighter_round1_kd,fight_fighter_round2_kd,fight_fighter_overall_sig_str_%,fight_fighter_round1_sig_str_%,fight_fighter_round2_sig_str_%,fight_fighter_overall_td_%,fight_fighter_round1_td_%,fight_fighter_round2_td_%,fight_fighter_overall_sub_att,fight_fighter_round1_sub_att,fight_fighter_round2_sub_att,fight_fighter_overall_rev,fight_fighter_round1_rev,fight_fighter_round2_rev,fight_fighter_overall_sig_str_%_ss,fight_fighter_round1_sig_str_%_ss,fight_fighter_round2_sig_str_%_ss,fight_fighter_round3_kd,fight_fighter_round3_sig_str_%,fight_fighter_round3_td_%,fight_fighter_round3_sub_att,fight_fighter_round3_rev,fight_fighter_round3_sig_str_%_ss,fight_fighter_round4_kd,fight_fighter_round4_sig_str_%,fight_fighter_round4_td_%,fight_fighter_round4_sub_att,fight_fighter_round4_rev,fight_fighter_round4_sig_str_%_ss,fight_fighter_round5_kd,fight_fighter_round5_sig_str_%,fight_fighter_round5_td_%,fight_fighter_round5_sub_att,fight_fighter_round5_rev,fight_fighter_round5_sig_str_%_ss,fight_weight_class,fight_fighter_won,event_id,fighter_id,fight_fighter_overall_sig_str_landed,fight_fighter_overall_sig_str_total,fight_fighter_round1_sig_str_landed,fight_fighter_round1_sig_str_total,fight_fighter_round2_sig_str_landed,fight_fighter_round2_sig_str_total,fight_fighter_overall_total_str_landed,fight_fighter_overall_total_str_total,fight_fighter_round1_total_str_landed,fight_fighter_round1_total_str_total,fight_fighter_round2_total_str_landed,fight_fighter_round2_total_str_total,fight_fighter_overall_td_landed,fight_fighter_overall_td_total,fight_fighter_round1_td_landed,fight_fighter_round1_td_total,fight_fighter_round2_td_landed,fight_fighter_round2_td_total,fight_fighter_overall_sig_str_ss_landed,fight_fighter_overall_sig_str_ss_total,fight_fighter_round1_sig_str_ss_landed,fight_fighter_round1_sig_str_ss_total,fight_fighter_round2_sig_str_ss_landed,fight_fighter_round2_sig_str_ss_total,fight_fighter_overall_head_ss_landed,fight_fighter_overall_head_ss_total,fight_fighter_round1_head_ss_landed,fight_fighter_round1_head_ss_total,fight_fighter_round2_head_ss_landed,fight_fighter_round2_head_ss_total,fight_fighter_overall_body_ss_landed,fight_fighter_overall_body_ss_total,fight_fighter_round1_body_ss_landed,fight_fighter_round1_body_ss_total,fight_fighter_round2_body_ss_landed,fight_fighter_round2_body_ss_total,fight_fighter_overall_leg_ss_landed,fight_fighter_overall_leg_ss_total,fight_fighter_round1_leg_ss_landed,fight_fighter_round1_leg_ss_total,fight_fighter_round2_leg_ss_landed,fight_fighter_round2_leg_ss_total,fight_fighter_overall_distance_ss_landed,fight_fighter_overall_distance_ss_total,fight_fighter_round1_distance_ss_landed,fight_fighter_round1_distance_ss_total,fight_fighter_round2_distance_ss_landed,fight_fighter_round2_distance_ss_total,fight_fighter_overall_clinch_ss_landed,fight_fighter_overall_clinch_ss_total,fight_fighter_round1_clinch_ss_landed,fight_fighter_round1_clinch_ss_total,fight_fighter_round2_clinch_ss_landed,fight_fighter_round2_clinch_ss_total,fight_fighter_overall_ground_ss_landed,fight_fighter_overall_ground_ss_total,fight_fighter_round1_ground_ss_landed,fight_fighter_round1_ground_ss_total,fight_fighter_round2_ground_ss_landed,fight_fighter_round2_ground_ss_total,fight_fighter_round3_sig_str_landed,fight_fighter_round3_sig_str_total,fight_fighter_round3_total_str_landed,fight_fighter_round3_total_str_total,fight_fighter_round3_td_landed,fight_fighter_round3_td_total,fight_fighter_round3_sig_str_ss_landed,fight_fighter_round3_sig_str_ss_total,fight_fighter_round3_head_ss_landed,fight_fighter_round3_head_ss_total,fight_fighter_round3_body_ss_landed,fight_fighter_round3_body_ss_total,fight_fighter_round3_leg_ss_landed,fight_fighter_round3_leg_ss_total,fight_fighter_round3_distance_ss_landed,fight_fighter_round3_distance_ss_total,fight_fighter_round3_clinch_ss_landed,fight_fighter_round3_clinch

In [13]:
df.shape

(13720, 174)